### Question 1: Getting the embeddings model

In [1]:
import warnings

warnings.filterwarnings("ignore")

# importing the sentence transformer library to convert text into vector embeddings
from sentence_transformers import SentenceTransformer

In [2]:
embedding_model = SentenceTransformer("multi-qa-distilbert-cos-v1")

In [3]:
# text that needs to be converted into a vector embeddings
user_question = "I just discovered the course. Can I still join it?"

In [4]:
# converting the text into a vector embedding using the encode method
v_q = embedding_model.encode(user_question)

In [5]:
# What's the first value of the resulting vector
v_q[0]

0.078222655

### Question 2: Create the embeddings

In [6]:
# prepare the documents - code block from homework assignment
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [7]:
# we only need a subset of of the questions - the questions for machine-learning-zoomcamp
documents_ml = [doc for doc in documents if doc['course'] == "machine-learning-zoomcamp"]

# homework mentioned that we need to have 375 records after filtering
len(documents_ml)

375

In [8]:
documents_ml[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

In [9]:
# create an embedding for both question and answer fields
embeddings = [embedding_model.encode(f"{doc['question']} {doc['text']}") for doc in documents_ml]

In [10]:
# shape of our embeddings
import numpy as np

X = np.array(embeddings)

X.shape

(375, 768)

### Question 3: Search

In [11]:
# vectors returned from emboding model are normalised, hence to compute the cosine similarity dot product would suffice
scores = X.dot(v_q)

In [12]:
# highest score in the results
scores.max()

0.6506573

In [13]:
# index of the result with the highest cosine similarity score
scores.argmax()

14

In [14]:
# this would be the response to our user query
print(documents_ml[scores.argmax()]['text'])

Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.


### Question 4 - Hit-rate for our search engine

In [15]:
# creating our search engine class
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings_array = embeddings

    def search(self, query_vector, num_of_res=10):
        scores = self.embeddings_array.dot(query_vector)
        idx = scores.argsort()[::-1][:num_of_res]

        return [self.documents[i] for i in idx]
    
      
search_engine = VectorSearchEngine(documents=documents_ml, embeddings=X)
search_engine.search(query_vector=v_q, num_of_res=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

In [16]:
# loading the ground truth dataset
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [32]:
from typing import List, Dict

# now to calculate the hit-rate - 
def hit_rate(dataset:List[Dict]):

    for data_dict in dataset:
        data_dict['results'] = search_engine.search(query_vector=embedding_model.encode(data_dict['question']), num_of_res=5)
        res_id = [res['id'] for res in data_dict['results']]
        if data_dict['document'] in res_id:
            data_dict['hit_rate'] = 1
        else:
            data_dict['hit_rate'] = 0

    hit_rate = [data_dict['hit_rate'] for data_dict in dataset]

    return sum(hit_rate) / len(hit_rate)

In [33]:
hit_rate(ground_truth)

0.9398907103825137

In [24]:
search_engine.search(embedding_model.encode(ground_truth[0]['question']), num_of_res=5)[0]['id']

'0a278fb2'

In [26]:
ground_truth[0]['document']

'0227b872'